In [0]:
#Getting all libraries necessary for now!
import pandas as pd
import numpy as np
from datetime import timedelta

import matplotlib.pyplot as plt
import seaborn as sns

import datetime

import warnings
warnings.filterwarnings('ignore')

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
# Read all these to pandas!
train = pd.read_csv("/content/drive/My Drive/LTFS/train_fwYjLYX.csv")
test = pd.read_csv("/content/drive/My Drive/LTFS/test_1eLl9Yf.csv")
submission = pd.read_csv("/content/drive/My Drive/LTFS/sample_submission_IIzFVsf.csv")

In [4]:
print(train.shape, test.shape)

(80402, 6) (180, 3)


In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80402 entries, 0 to 80401
Data columns (total 6 columns):
application_date    80402 non-null object
segment             80402 non-null int64
branch_id           66898 non-null float64
state               80402 non-null object
zone                66898 non-null object
case_count          80402 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 3.7+ MB


In [0]:
#Here we are not having any unique field! We need to find proper features! Huge task ahead
#As the train and test data are not uniform, the train data should be transformed to the format of test data without losing much data
#All we need is case count for each day! Need to consider holidays too!

In [0]:
train['application_date'] = pd.to_datetime(train['application_date'])
test['application_date'] = pd.to_datetime(test['application_date'])

In [8]:
import holidays
train = train.sort_values('application_date')
train.head()

,application_date,segment,branch_id,state,zone,case_count
0,2017-04-01,1,1.0,WEST BENGAL,EAST,40.0
20150,2017-04-01,1,62.0,PUNJAB,NORTH,2.0
20956,2017-04-01,1,63.0,JHARKHAND,SOUTH,6.0
21762,2017-04-01,1,64.0,KARNATAKA,SOUTH,0.0
22568,2017-04-01,1,65.0,ASSAM,EAST,12.0


In [9]:
train = train.reset_index(drop=True)
train.head(100)

,application_date,segment,branch_id,state,zone,case_count
0,2017-04-01,1,1.0,WEST BENGAL,EAST,40.0
1,2017-04-01,1,62.0,PUNJAB,NORTH,2.0
2,2017-04-01,1,63.0,JHARKHAND,SOUTH,6.0
3,2017-04-01,1,64.0,KARNATAKA,SOUTH,0.0
4,2017-04-01,1,65.0,ASSAM,EAST,12.0
...,...,...,...,...,...,...
95,2017-04-01,1,250.0,GUJARAT,WEST,0.0
96,2017-04-01,2,NaN,KARNATAKA,NaN,165.0
97,2017-04-01,1,3.0,DELHI,NORTH,58.0
98,2017-04-01,2,NaN,HARYANA,NaN,0.0


In [10]:
#Splitting the dataframe into Segment 1 and 2
train1 = train[train['segment']==1]
train2 = train[train['segment']==2]
train1 = train1.reset_index(drop=True)
train2 = train2.reset_index(drop=True)
print(train1.head())
print(train2.head())

  application_date  segment  branch_id        state   zone  case_count
0       2017-04-01        1        1.0  WEST BENGAL   EAST        40.0
1       2017-04-01        1       62.0       PUNJAB  NORTH         2.0
2       2017-04-01        1       63.0    JHARKHAND  SOUTH         6.0
3       2017-04-01        1       64.0    KARNATAKA  SOUTH         0.0
4       2017-04-01        1       65.0        ASSAM   EAST        12.0
  application_date  segment  branch_id          state zone  case_count
0       2017-04-01        2        NaN     TAMIL NADU  NaN       458.0
1       2017-04-01        2        NaN         PUNJAB  NaN         0.0
2       2017-04-01        2        NaN        TRIPURA  NaN         0.0
3       2017-04-01        2        NaN    WEST BENGAL  NaN        43.0
4       2017-04-01        2        NaN  UTTAR PRADESH  NaN        27.0


In [11]:
date_train1 = train1.drop(['branch_id','state','zone'], axis=1)
date_train2 = train2.drop(['branch_id','state','zone'], axis=1)
print(date_train1.head())
print(date_train2.head())

  application_date  segment  case_count
0       2017-04-01        1        40.0
1       2017-04-01        1         2.0
2       2017-04-01        1         6.0
3       2017-04-01        1         0.0
4       2017-04-01        1        12.0
  application_date  segment  case_count
0       2017-04-01        2       458.0
1       2017-04-01        2         0.0
2       2017-04-01        2         0.0
3       2017-04-01        2        43.0
4       2017-04-01        2        27.0


In [12]:
date_train1 = date_train1.groupby('application_date',as_index= False).agg({'case_count':'sum'})
date_train1['segment'] = 1
date_train1 = date_train1[['application_date','segment','case_count']]
date_train1.head()

,application_date,segment,case_count
0,2017-04-01,1,299.0
1,2017-04-03,1,42.0
2,2017-04-04,1,23.0
3,2017-04-05,1,1530.0
4,2017-04-07,1,1341.0


In [13]:
date_train2 = date_train2.groupby('application_date',as_index= False).agg({'case_count':'sum'})
date_train2['segment'] = 2
date_train2 = date_train2[['application_date','segment','case_count']]
date_train2.head()

,application_date,segment,case_count
0,2017-04-01,2,897.0
1,2017-04-02,2,605.0
2,2017-04-03,2,2016.0
3,2017-04-04,2,2245.0
4,2017-04-05,2,2415.0


In [14]:
date_train1.describe() # There are outliers, as expected!

,segment,case_count
count,806.0,806.000000
mean,1.0,2714.689826
std,0.0,2042.418090
min,1.0,1.000000
25%,1.0,1633.250000
50%,1.0,2602.000000
75%,1.0,3333.000000
max,1.0,21885.000000


In [15]:
date_train2.describe()

,segment,case_count
count,844.0,844.000000
mean,2.0,15077.098341
std,0.0,10188.900682
min,2.0,206.000000
25%,2.0,5946.000000
50%,2.0,12679.000000
75%,2.0,24217.250000
max,2.0,43340.000000


In [0]:
test1 = test[test['segment']==1]
test2 = test[test['segment']==2]

In [17]:
test1.drop('id', axis=1, inplace = True)
test2.drop('id', axis=1, inplace = True)
print(test1.head())
print(test2.head())

  application_date  segment
0       2019-07-06        1
1       2019-07-07        1
2       2019-07-08        1
3       2019-07-09        1
4       2019-07-10        1
   application_date  segment
87       2019-07-24        2
88       2019-07-25        2
89       2019-07-26        2
90       2019-07-27        2
91       2019-07-28        2


In [18]:
print(date_train1.shape, date_train2.shape, test1.shape, test2.shape)

(806, 3) (844, 3) (87, 2) (93, 2)


In [19]:
df1 = date_train1.append(test1, ignore_index = True)
df2 = date_train2.append(test2, ignore_index = True)
print(df1.shape)
print(df2.shape)
print(df1.head())
print(df2.head())

(893, 3)
(937, 3)
  application_date  case_count  segment
0       2017-04-01       299.0        1
1       2017-04-03        42.0        1
2       2017-04-04        23.0        1
3       2017-04-05      1530.0        1
4       2017-04-07      1341.0        1
  application_date  case_count  segment
0       2017-04-01       897.0        2
1       2017-04-02       605.0        2
2       2017-04-03      2016.0        2
3       2017-04-04      2245.0        2
4       2017-04-05      2415.0        2


In [20]:
train1['state_zone'] = train1[['zone','state']].apply(lambda x: ' '.join(x), axis=1)
train1.head()

,application_date,segment,branch_id,state,zone,case_count,state_zone
0,2017-04-01,1,1.0,WEST BENGAL,EAST,40.0,EAST WEST BENGAL
1,2017-04-01,1,62.0,PUNJAB,NORTH,2.0,NORTH PUNJAB
2,2017-04-01,1,63.0,JHARKHAND,SOUTH,6.0,SOUTH JHARKHAND
3,2017-04-01,1,64.0,KARNATAKA,SOUTH,0.0,SOUTH KARNATAKA
4,2017-04-01,1,65.0,ASSAM,EAST,12.0,EAST ASSAM


In [21]:
print(train1['state'].value_counts())
print(train1['state_zone'].value_counts()) # Orissa alone is South and East Orissa

MAHARASHTRA       10478
GUJARAT            6448
WEST BENGAL        6448
TAMIL NADU         5642
UTTAR PRADESH      5642
ANDHRA PRADESH     4836
PUNJAB             4030
ASSAM              3224
HARYANA            3224
KARNATAKA          2418
BIHAR              2418
ORISSA             2418
TELANGANA          1612
TRIPURA            1612
UTTARAKHAND        1612
JHARKHAND          1612
KERALA              806
MADHYA PRADESH      806
CHHATTISGARH        806
DELHI               806
Name: state, dtype: int64
WEST MAHARASHTRA          10478
WEST GUJARAT               6448
EAST WEST BENGAL           6448
EAST UTTAR PRADESH         5642
SOUTH TAMIL NADU           5642
SOUTH ANDHRA PRADESH       4836
NORTH PUNJAB               4030
NORTH HARYANA              3224
EAST ASSAM                 3224
EAST BIHAR                 2418
SOUTH KARNATAKA            2418
SOUTH TELANGANA            1612
EAST UTTARAKHAND           1612
EAST TRIPURA               1612
SOUTH JHARKHAND            1612
SOUTH ORISSA  

In [22]:
state_count = train1.groupby(['application_date'])['state'].count().reset_index()
state_count.rename(columns = {'state' : 'state_count'}, inplace = True)
df1 = df1.merge(state_count, how = 'left', on = ['application_date'])
unique_state = train1.groupby(['application_date'])['state'].nunique().reset_index()
unique_state.rename(columns = {'state' : 'unique_state'}, inplace = True)
df1 = df1.merge(unique_state, how = 'left', on = ['application_date'])
df1.head()

,application_date,case_count,segment,state_count,unique_state
0,2017-04-01,299.0,1,83.0,20.0
1,2017-04-03,42.0,1,83.0,20.0
2,2017-04-04,23.0,1,83.0,20.0
3,2017-04-05,1530.0,1,83.0,20.0
4,2017-04-07,1341.0,1,83.0,20.0


In [23]:
unique_state_zone = train1.groupby(['application_date'])['state_zone'].nunique().reset_index()
unique_state_zone.rename(columns = {'state_zone' : 'unique_state_zone'}, inplace = True)
df1 = df1.merge(unique_state_zone, how = 'left', on = ['application_date'])
df1.head()

,application_date,case_count,segment,state_count,unique_state,unique_state_zone
0,2017-04-01,299.0,1,83.0,20.0,21.0
1,2017-04-03,42.0,1,83.0,20.0,21.0
2,2017-04-04,23.0,1,83.0,20.0,21.0
3,2017-04-05,1530.0,1,83.0,20.0,21.0
4,2017-04-07,1341.0,1,83.0,20.0,21.0


In [24]:
dummy_train1 = pd.get_dummies(train1, columns=['state','state_zone'])
dummy_train1.head(200)

,application_date,segment,branch_id,zone,case_count,state_ANDHRA PRADESH,state_ASSAM,state_BIHAR,state_CHHATTISGARH,state_DELHI,state_GUJARAT,state_HARYANA,state_JHARKHAND,state_KARNATAKA,state_KERALA,state_MADHYA PRADESH,state_MAHARASHTRA,state_ORISSA,state_PUNJAB,state_TAMIL NADU,state_TELANGANA,state_TRIPURA,state_UTTAR PRADESH,state_UTTARAKHAND,state_WEST BENGAL,state_zone_CENTRAL CHHATTISGARH,state_zone_CENTRAL MADHYA PRADESH,state_zone_EAST ASSAM,state_zone_EAST BIHAR,state_zone_EAST ORISSA,state_zone_EAST TRIPURA,state_zone_EAST UTTAR PRADESH,state_zone_EAST UTTARAKHAND,state_zone_EAST WEST BENGAL,state_zone_NORTH DELHI,state_zone_NORTH HARYANA,state_zone_NORTH PUNJAB,state_zone_SOUTH ANDHRA PRADESH,state_zone_SOUTH JHARKHAND,state_zone_SOUTH KARNATAKA,state_zone_SOUTH KERALA,state_zone_SOUTH ORISSA,state_zone_SOUTH TAMIL NADU,state_zone_SOUTH TELANGANA,state_zone_WEST GUJARAT,state_zone_WEST MAHARASHTRA
0,2017-04-01,1,1.0,EAST,40.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,2017-04-01,1,62.0,NORTH,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,2017-04-01,1,63.0,SOUTH,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,2017-04-01,1,64.0,SOUTH,0.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,2017-04-01,1,65.0,EAST,12.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2017-04-04,1,159.0,EAST,0.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
196,2017-04-04,1,100.0,EAST,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
197,2017-04-04,1,117.0,WEST,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
198,2017-04-04,1,68.0,WEST,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [25]:
dummy_train1.drop('zone', axis=1, inplace=True)
dummy_train1.head()

,application_date,segment,branch_id,case_count,state_ANDHRA PRADESH,state_ASSAM,state_BIHAR,state_CHHATTISGARH,state_DELHI,state_GUJARAT,state_HARYANA,state_JHARKHAND,state_KARNATAKA,state_KERALA,state_MADHYA PRADESH,state_MAHARASHTRA,state_ORISSA,state_PUNJAB,state_TAMIL NADU,state_TELANGANA,state_TRIPURA,state_UTTAR PRADESH,state_UTTARAKHAND,state_WEST BENGAL,state_zone_CENTRAL CHHATTISGARH,state_zone_CENTRAL MADHYA PRADESH,state_zone_EAST ASSAM,state_zone_EAST BIHAR,state_zone_EAST ORISSA,state_zone_EAST TRIPURA,state_zone_EAST UTTAR PRADESH,state_zone_EAST UTTARAKHAND,state_zone_EAST WEST BENGAL,state_zone_NORTH DELHI,state_zone_NORTH HARYANA,state_zone_NORTH PUNJAB,state_zone_SOUTH ANDHRA PRADESH,state_zone_SOUTH JHARKHAND,state_zone_SOUTH KARNATAKA,state_zone_SOUTH KERALA,state_zone_SOUTH ORISSA,state_zone_SOUTH TAMIL NADU,state_zone_SOUTH TELANGANA,state_zone_WEST GUJARAT,state_zone_WEST MAHARASHTRA
0,2017-04-01,1,1.0,40.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,2017-04-01,1,62.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,2017-04-01,1,63.0,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,2017-04-01,1,64.0,0.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,2017-04-01,1,65.0,12.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [26]:
agg_col={}
for k in dummy_train1.columns:
    if k.startswith('state') or k.startswith('state_zone'):
        agg_col[k]= 'sum'
agg_col

{'state_ANDHRA PRADESH': 'sum',
 'state_ASSAM': 'sum',
 'state_BIHAR': 'sum',
 'state_CHHATTISGARH': 'sum',
 'state_DELHI': 'sum',
 'state_GUJARAT': 'sum',
 'state_HARYANA': 'sum',
 'state_JHARKHAND': 'sum',
 'state_KARNATAKA': 'sum',
 'state_KERALA': 'sum',
 'state_MADHYA PRADESH': 'sum',
 'state_MAHARASHTRA': 'sum',
 'state_ORISSA': 'sum',
 'state_PUNJAB': 'sum',
 'state_TAMIL NADU': 'sum',
 'state_TELANGANA': 'sum',
 'state_TRIPURA': 'sum',
 'state_UTTAR PRADESH': 'sum',
 'state_UTTARAKHAND': 'sum',
 'state_WEST BENGAL': 'sum',
 'state_zone_CENTRAL CHHATTISGARH': 'sum',
 'state_zone_CENTRAL MADHYA PRADESH': 'sum',
 'state_zone_EAST ASSAM': 'sum',
 'state_zone_EAST BIHAR': 'sum',
 'state_zone_EAST ORISSA': 'sum',
 'state_zone_EAST TRIPURA': 'sum',
 'state_zone_EAST UTTAR PRADESH': 'sum',
 'state_zone_EAST UTTARAKHAND': 'sum',
 'state_zone_EAST WEST BENGAL': 'sum',
 'state_zone_NORTH DELHI': 'sum',
 'state_zone_NORTH HARYANA': 'sum',
 'state_zone_NORTH PUNJAB': 'sum',
 'state_zone_SOU

In [27]:
each_state_count = dummy_train1.groupby('application_date').agg(agg_col)
each_state_count.head()

,state_ANDHRA PRADESH,state_ASSAM,state_BIHAR,state_CHHATTISGARH,state_DELHI,state_GUJARAT,state_HARYANA,state_JHARKHAND,state_KARNATAKA,state_KERALA,state_MADHYA PRADESH,state_MAHARASHTRA,state_ORISSA,state_PUNJAB,state_TAMIL NADU,state_TELANGANA,state_TRIPURA,state_UTTAR PRADESH,state_UTTARAKHAND,state_WEST BENGAL,state_zone_CENTRAL CHHATTISGARH,state_zone_CENTRAL MADHYA PRADESH,state_zone_EAST ASSAM,state_zone_EAST BIHAR,state_zone_EAST ORISSA,state_zone_EAST TRIPURA,state_zone_EAST UTTAR PRADESH,state_zone_EAST UTTARAKHAND,state_zone_EAST WEST BENGAL,state_zone_NORTH DELHI,state_zone_NORTH HARYANA,state_zone_NORTH PUNJAB,state_zone_SOUTH ANDHRA PRADESH,state_zone_SOUTH JHARKHAND,state_zone_SOUTH KARNATAKA,state_zone_SOUTH KERALA,state_zone_SOUTH ORISSA,state_zone_SOUTH TAMIL NADU,state_zone_SOUTH TELANGANA,state_zone_WEST GUJARAT,state_zone_WEST MAHARASHTRA
application_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-04-01,6,4,3,1,1,8,4,2,3,1,1,13,3,5,7,2,2,7,2,8,1,1,4,3,1,2,7,2,8,1,4,5,6,2,3,1,2,7,2,8,13
2017-04-03,6,4,3,1,1,8,4,2,3,1,1,13,3,5,7,2,2,7,2,8,1,1,4,3,1,2,7,2,8,1,4,5,6,2,3,1,2,7,2,8,13
2017-04-04,6,4,3,1,1,8,4,2,3,1,1,13,3,5,7,2,2,7,2,8,1,1,4,3,1,2,7,2,8,1,4,5,6,2,3,1,2,7,2,8,13
2017-04-05,6,4,3,1,1,8,4,2,3,1,1,13,3,5,7,2,2,7,2,8,1,1,4,3,1,2,7,2,8,1,4,5,6,2,3,1,2,7,2,8,13
2017-04-07,6,4,3,1,1,8,4,2,3,1,1,13,3,5,7,2,2,7,2,8,1,1,4,3,1,2,7,2,8,1,4,5,6,2,3,1,2,7,2,8,13


In [28]:
dummy_nonzero_train1 = dummy_train1[dummy_train1['case_count']!=0]
dummy_nonzero_train1.reset_index(inplace=True)
dummy_nonzero_train1.drop('index', axis=1, inplace= True)
dummy_nonzero_train1

,application_date,segment,branch_id,case_count,state_ANDHRA PRADESH,state_ASSAM,state_BIHAR,state_CHHATTISGARH,state_DELHI,state_GUJARAT,state_HARYANA,state_JHARKHAND,state_KARNATAKA,state_KERALA,state_MADHYA PRADESH,state_MAHARASHTRA,state_ORISSA,state_PUNJAB,state_TAMIL NADU,state_TELANGANA,state_TRIPURA,state_UTTAR PRADESH,state_UTTARAKHAND,state_WEST BENGAL,state_zone_CENTRAL CHHATTISGARH,state_zone_CENTRAL MADHYA PRADESH,state_zone_EAST ASSAM,state_zone_EAST BIHAR,state_zone_EAST ORISSA,state_zone_EAST TRIPURA,state_zone_EAST UTTAR PRADESH,state_zone_EAST UTTARAKHAND,state_zone_EAST WEST BENGAL,state_zone_NORTH DELHI,state_zone_NORTH HARYANA,state_zone_NORTH PUNJAB,state_zone_SOUTH ANDHRA PRADESH,state_zone_SOUTH JHARKHAND,state_zone_SOUTH KARNATAKA,state_zone_SOUTH KERALA,state_zone_SOUTH ORISSA,state_zone_SOUTH TAMIL NADU,state_zone_SOUTH TELANGANA,state_zone_WEST GUJARAT,state_zone_WEST MAHARASHTRA
0,2017-04-01,1,1.0,40.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,2017-04-01,1,62.0,2.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,2017-04-01,1,63.0,6.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
3,2017-04-01,1,65.0,12.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,2017-04-01,1,67.0,8.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47409,2019-07-05,1,61.0,1.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
47410,2019-07-05,1,137.0,5.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
47411,2019-07-05,1,135.0,2.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
47412,2019-07-05,1,18.0,1.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [29]:
agg_col={}
for k in dummy_nonzero_train1.columns:
    if k.startswith('state') or k.startswith('state_zone'):
        agg_col[k]= 'sum'
agg_col

{'state_ANDHRA PRADESH': 'sum',
 'state_ASSAM': 'sum',
 'state_BIHAR': 'sum',
 'state_CHHATTISGARH': 'sum',
 'state_DELHI': 'sum',
 'state_GUJARAT': 'sum',
 'state_HARYANA': 'sum',
 'state_JHARKHAND': 'sum',
 'state_KARNATAKA': 'sum',
 'state_KERALA': 'sum',
 'state_MADHYA PRADESH': 'sum',
 'state_MAHARASHTRA': 'sum',
 'state_ORISSA': 'sum',
 'state_PUNJAB': 'sum',
 'state_TAMIL NADU': 'sum',
 'state_TELANGANA': 'sum',
 'state_TRIPURA': 'sum',
 'state_UTTAR PRADESH': 'sum',
 'state_UTTARAKHAND': 'sum',
 'state_WEST BENGAL': 'sum',
 'state_zone_CENTRAL CHHATTISGARH': 'sum',
 'state_zone_CENTRAL MADHYA PRADESH': 'sum',
 'state_zone_EAST ASSAM': 'sum',
 'state_zone_EAST BIHAR': 'sum',
 'state_zone_EAST ORISSA': 'sum',
 'state_zone_EAST TRIPURA': 'sum',
 'state_zone_EAST UTTAR PRADESH': 'sum',
 'state_zone_EAST UTTARAKHAND': 'sum',
 'state_zone_EAST WEST BENGAL': 'sum',
 'state_zone_NORTH DELHI': 'sum',
 'state_zone_NORTH HARYANA': 'sum',
 'state_zone_NORTH PUNJAB': 'sum',
 'state_zone_SOU

In [30]:
each_state_nonzero = dummy_nonzero_train1.groupby('application_date').agg(agg_col)
each_state_nonzero.head()

,state_ANDHRA PRADESH,state_ASSAM,state_BIHAR,state_CHHATTISGARH,state_DELHI,state_GUJARAT,state_HARYANA,state_JHARKHAND,state_KARNATAKA,state_KERALA,state_MADHYA PRADESH,state_MAHARASHTRA,state_ORISSA,state_PUNJAB,state_TAMIL NADU,state_TELANGANA,state_TRIPURA,state_UTTAR PRADESH,state_UTTARAKHAND,state_WEST BENGAL,state_zone_CENTRAL CHHATTISGARH,state_zone_CENTRAL MADHYA PRADESH,state_zone_EAST ASSAM,state_zone_EAST BIHAR,state_zone_EAST ORISSA,state_zone_EAST TRIPURA,state_zone_EAST UTTAR PRADESH,state_zone_EAST UTTARAKHAND,state_zone_EAST WEST BENGAL,state_zone_NORTH DELHI,state_zone_NORTH HARYANA,state_zone_NORTH PUNJAB,state_zone_SOUTH ANDHRA PRADESH,state_zone_SOUTH JHARKHAND,state_zone_SOUTH KARNATAKA,state_zone_SOUTH KERALA,state_zone_SOUTH ORISSA,state_zone_SOUTH TAMIL NADU,state_zone_SOUTH TELANGANA,state_zone_WEST GUJARAT,state_zone_WEST MAHARASHTRA
application_date,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2017-04-01,0,2,1,0,1,4,0,2,1,0,1,4,3,4,2,1,1,0,1,6,0,1,2,1,1,1,0,1,6,1,0,4,0,2,1,0,2,2,1,4,4
2017-04-03,0,1,0,0,1,0,0,0,1,0,1,2,1,3,0,0,0,0,1,4,0,1,1,0,0,0,0,1,4,1,0,3,0,0,1,0,1,0,0,0,2
2017-04-04,0,0,0,0,1,0,0,0,1,0,1,3,0,0,0,0,0,0,0,3,0,1,0,0,0,0,0,0,3,1,0,0,0,0,1,0,0,0,0,0,3
2017-04-05,1,2,1,0,1,5,0,2,2,1,1,9,3,4,3,1,1,0,1,8,0,1,2,1,1,1,0,1,8,1,0,4,1,2,2,1,2,3,1,5,9
2017-04-07,1,2,1,0,1,5,0,2,2,1,1,9,3,4,3,1,1,0,1,8,0,1,2,1,1,1,0,1,8,1,0,4,1,2,2,1,2,3,1,5,9


In [31]:
each_state_count.shape
each_state_nonzero.shape

(806, 41)

In [32]:
df1 = df1.merge(each_state_count, how = 'left', on = ['application_date'])
df1 = df1.merge(each_state_nonzero, how = 'left', on = ['application_date'])
df1.head()

,application_date,case_count,segment,state_count,unique_state,unique_state_zone,state_ANDHRA PRADESH_x,state_ASSAM_x,state_BIHAR_x,state_CHHATTISGARH_x,state_DELHI_x,state_GUJARAT_x,state_HARYANA_x,state_JHARKHAND_x,state_KARNATAKA_x,state_KERALA_x,state_MADHYA PRADESH_x,state_MAHARASHTRA_x,state_ORISSA_x,state_PUNJAB_x,state_TAMIL NADU_x,state_TELANGANA_x,state_TRIPURA_x,state_UTTAR PRADESH_x,state_UTTARAKHAND_x,state_WEST BENGAL_x,state_zone_CENTRAL CHHATTISGARH_x,state_zone_CENTRAL MADHYA PRADESH_x,state_zone_EAST ASSAM_x,state_zone_EAST BIHAR_x,state_zone_EAST ORISSA_x,state_zone_EAST TRIPURA_x,state_zone_EAST UTTAR PRADESH_x,state_zone_EAST UTTARAKHAND_x,state_zone_EAST WEST BENGAL_x,state_zone_NORTH DELHI_x,state_zone_NORTH HARYANA_x,state_zone_NORTH PUNJAB_x,state_zone_SOUTH ANDHRA PRADESH_x,state_zone_SOUTH JHARKHAND_x,state_zone_SOUTH KARNATAKA_x,state_zone_SOUTH KERALA_x,state_zone_SOUTH ORISSA_x,state_zone_SOUTH TAMIL NADU_x,state_zone_SOUTH TELANGANA_x,state_zone_WEST GUJARAT_x,state_zone_WEST MAHARASHTRA_x,state_ANDHRA PRADESH_y,state_ASSAM_y,state_BIHAR_y,state_CHHATTISGARH_y,state_DELHI_y,state_GUJARAT_y,state_HARYANA_y,state_JHARKHAND_y,state_KARNATAKA_y,state_KERALA_y,state_MADHYA PRADESH_y,state_MAHARASHTRA_y,state_ORISSA_y,state_PUNJAB_y,state_TAMIL NADU_y,state_TELANGANA_y,state_TRIPURA_y,state_UTTAR PRADESH_y,state_UTTARAKHAND_y,state_WEST BENGAL_y,state_zone_CENTRAL CHHATTISGARH_y,state_zone_CENTRAL MADHYA PRADESH_y,state_zone_EAST ASSAM_y,state_zone_EAST BIHAR_y,state_zone_EAST ORISSA_y,state_zone_EAST TRIPURA_y,state_zone_EAST UTTAR PRADESH_y,state_zone_EAST UTTARAKHAND_y,state_zone_EAST WEST BENGAL_y,state_zone_NORTH DELHI_y,state_zone_NORTH HARYANA_y,state_zone_NORTH PUNJAB_y,state_zone_SOUTH ANDHRA PRADESH_y,state_zone_SOUTH JHARKHAND_y,state_zone_SOUTH KARNATAKA_y,state_zone_SOUTH KERALA_y,state_zone_SOUTH ORISSA_y,state_zone_SOUTH TAMIL NADU_y,state_zone_SOUTH TELANGANA_y,state_zone_WEST GUJARAT_y,state_zone_WEST MAHARASHTRA_y
0,2017-04-01,299.0,1,83.0,20.0,21.0,6.0,4.0,3.0,1.0,1.0,8.0,4.0,2.0,3.0,1.0,1.0,13.0,3.0,5.0,7.0,2.0,2.0,7.0,2.0,8.0,1.0,1.0,4.0,3.0,1.0,2.0,7.0,2.0,8.0,1.0,4.0,5.0,6.0,2.0,3.0,1.0,2.0,7.0,2.0,8.0,13.0,0.0,2.0,1.0,0.0,1.0,4.0,0.0,2.0,1.0,0.0,1.0,4.0,3.0,4.0,2.0,1.0,1.0,0.0,1.0,6.0,0.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,6.0,1.0,0.0,4.0,0.0,2.0,1.0,0.0,2.0,2.0,1.0,4.0,4.0
1,2017-04-03,42.0,1,83.0,20.0,21.0,6.0,4.0,3.0,1.0,1.0,8.0,4.0,2.0,3.0,1.0,1.0,13.0,3.0,5.0,7.0,2.0,2.0,7.0,2.0,8.0,1.0,1.0,4.0,3.0,1.0,2.0,7.0,2.0,8.0,1.0,4.0,5.0,6.0,2.0,3.0,1.0,2.0,7.0,2.0,8.0,13.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0
2,2017-04-04,23.0,1,83.0,20.0,21.0,6.0,4.0,3.0,1.0,1.0,8.0,4.0,2.0,3.0,1.0,1.0,13.0,3.0,5.0,7.0,2.0,2.0,7.0,2.0,8.0,1.0,1.0,4.0,3.0,1.0,2.0,7.0,2.0,8.0,1.0,4.0,5.0,6.0,2.0,3.0,1.0,2.0,7.0,2.0,8.0,13.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0
3,2017-04-05,1530.0,1,83.0,20.0,21.0,6.0,4.0,3.0,1.0,1.0,8.0,4.0,2.0,3.0,1.0,1.0,13.0,3.0,5.0,7.0,2.0,2.0,7.0,2.0,8.0,1.0,1.0,4.0,3.0,1.0,2.0,7.0,2.0,8.0,1.0,4.0,5.0,6.0,2.0,3.0,1.0,2.0,7.0,2.0,8.0,13.0,1.0,2.0,1.0,0.0,1.0,5.0,0.0,2.0,2.0,1.0,1.0,9.0,3.0,4.0,3.0,1.0,1.0,0.0,1.0,8.0,0.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,8.0,1.0,0.0,4.0,1.0,2.0,2.0,1.0,2.0,3.0,1.0,5.0,9.0
4,2017-04-07,1341.0,1,83.0,20.0,21.0,6.0,4.0,3.0,1.0,1.0,8.0,4.0,2.0,3.0,1.0,1.0,13.0,3.0,5.0,7.0,2.0,2.0,7.0,2.0,8.0,1.0,1.0,4.0,3.0,1.0,2.0,7.0,2.0,8.0,1.0,4.0,5.0,6.0,2.0,3.0,1.0,2.0,7.0,2.0,8.0,13.0,1.0,2.0,1.0,0.0,1.0,5.0,0.0,2.0,2.0,1.0,1.0,9.0,3.0,4.0,3.0,1.0,1.0,0.0,1.0,8.0,0.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,8.0,1.0,0.0,4.0,1.0,2.0,2.0,1.0,2.0,3.0,1.0,5.0,9.0


In [0]:
from fastai.tabular import add_datepart

add_datepart(df1, 'application_date', drop=False)

df1.drop('application_Elapsed', axis=1, inplace=True)

In [34]:
df1.shape

(893, 100)

In [35]:
# Now for df2 (business segment 2)
state_count = train2.groupby(['application_date'])['state'].count().reset_index()
state_count.rename(columns = {'state' : 'state_count'}, inplace = True)
df2 = df2.merge(state_count, how = 'left', on = ['application_date'])
df2.head()

,application_date,case_count,segment,state_count
0,2017-04-01,897.0,2,16.0
1,2017-04-02,605.0,2,16.0
2,2017-04-03,2016.0,2,16.0
3,2017-04-04,2245.0,2,16.0
4,2017-04-05,2415.0,2,16.0


In [36]:
dummy_train2 = pd.get_dummies(train2, columns=['state'])
dummy_train2.head(200)

,application_date,segment,branch_id,zone,case_count,state_ASSAM,state_BIHAR,state_CHHATTISGARH,state_GUJARAT,state_HARYANA,state_JHARKHAND,state_KARNATAKA,state_KERALA,state_MADHYA PRADESH,state_MAHARASHTRA,state_ORISSA,state_PUNJAB,state_TAMIL NADU,state_TRIPURA,state_UTTAR PRADESH,state_WEST BENGAL
0,2017-04-01,2,NaN,NaN,458.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,2017-04-01,2,NaN,NaN,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,2017-04-01,2,NaN,NaN,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,2017-04-01,2,NaN,NaN,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,2017-04-01,2,NaN,NaN,27.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,2017-04-13,2,NaN,NaN,2340.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
196,2017-04-13,2,NaN,NaN,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
197,2017-04-13,2,NaN,NaN,755.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
198,2017-04-13,2,NaN,NaN,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [37]:
dummy_train2.drop(['branch_id','zone'], axis=1, inplace=True)
dummy_train2.head()

,application_date,segment,case_count,state_ASSAM,state_BIHAR,state_CHHATTISGARH,state_GUJARAT,state_HARYANA,state_JHARKHAND,state_KARNATAKA,state_KERALA,state_MADHYA PRADESH,state_MAHARASHTRA,state_ORISSA,state_PUNJAB,state_TAMIL NADU,state_TRIPURA,state_UTTAR PRADESH,state_WEST BENGAL
0,2017-04-01,2,458.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,2017-04-01,2,0.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,2017-04-01,2,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,2017-04-01,2,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,2017-04-01,2,27.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [38]:
agg_col={}
for k in dummy_train2.columns:
    if k.startswith('state'):
        agg_col[k]= 'sum'
agg_col

{'state_ASSAM': 'sum',
 'state_BIHAR': 'sum',
 'state_CHHATTISGARH': 'sum',
 'state_GUJARAT': 'sum',
 'state_HARYANA': 'sum',
 'state_JHARKHAND': 'sum',
 'state_KARNATAKA': 'sum',
 'state_KERALA': 'sum',
 'state_MADHYA PRADESH': 'sum',
 'state_MAHARASHTRA': 'sum',
 'state_ORISSA': 'sum',
 'state_PUNJAB': 'sum',
 'state_TAMIL NADU': 'sum',
 'state_TRIPURA': 'sum',
 'state_UTTAR PRADESH': 'sum',
 'state_WEST BENGAL': 'sum'}

In [39]:
each_state_count = dummy_train2.groupby('application_date').agg(agg_col)
each_state_count.head()

,state_ASSAM,state_BIHAR,state_CHHATTISGARH,state_GUJARAT,state_HARYANA,state_JHARKHAND,state_KARNATAKA,state_KERALA,state_MADHYA PRADESH,state_MAHARASHTRA,state_ORISSA,state_PUNJAB,state_TAMIL NADU,state_TRIPURA,state_UTTAR PRADESH,state_WEST BENGAL
application_date,,,,,,,,,,,,,,,,
2017-04-01,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2017-04-02,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2017-04-03,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2017-04-04,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1
2017-04-05,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1


In [40]:
dummy_nonzero_train2 = dummy_train2[dummy_train2['case_count']!=0]
dummy_nonzero_train2.reset_index(inplace=True)
dummy_nonzero_train2.drop('index', axis=1, inplace= True)
dummy_nonzero_train2

,application_date,segment,case_count,state_ASSAM,state_BIHAR,state_CHHATTISGARH,state_GUJARAT,state_HARYANA,state_JHARKHAND,state_KARNATAKA,state_KERALA,state_MADHYA PRADESH,state_MAHARASHTRA,state_ORISSA,state_PUNJAB,state_TAMIL NADU,state_TRIPURA,state_UTTAR PRADESH,state_WEST BENGAL
0,2017-04-01,2,458.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
1,2017-04-01,2,43.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2017-04-01,2,27.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,2017-04-01,2,111.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
4,2017-04-01,2,23.0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10776,2019-07-23,2,6180.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
10777,2019-07-23,2,518.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
10778,2019-07-23,2,457.0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
10779,2019-07-23,2,1489.0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [41]:
agg_col={}
for k in dummy_nonzero_train2.columns:
    if k.startswith('state'):
        agg_col[k]= 'sum'
agg_col

{'state_ASSAM': 'sum',
 'state_BIHAR': 'sum',
 'state_CHHATTISGARH': 'sum',
 'state_GUJARAT': 'sum',
 'state_HARYANA': 'sum',
 'state_JHARKHAND': 'sum',
 'state_KARNATAKA': 'sum',
 'state_KERALA': 'sum',
 'state_MADHYA PRADESH': 'sum',
 'state_MAHARASHTRA': 'sum',
 'state_ORISSA': 'sum',
 'state_PUNJAB': 'sum',
 'state_TAMIL NADU': 'sum',
 'state_TRIPURA': 'sum',
 'state_UTTAR PRADESH': 'sum',
 'state_WEST BENGAL': 'sum'}

In [42]:
each_state_nonzero = dummy_nonzero_train2.groupby('application_date').agg(agg_col)
each_state_nonzero.head()

,state_ASSAM,state_BIHAR,state_CHHATTISGARH,state_GUJARAT,state_HARYANA,state_JHARKHAND,state_KARNATAKA,state_KERALA,state_MADHYA PRADESH,state_MAHARASHTRA,state_ORISSA,state_PUNJAB,state_TAMIL NADU,state_TRIPURA,state_UTTAR PRADESH,state_WEST BENGAL
application_date,,,,,,,,,,,,,,,,
2017-04-01,0,0,1,0,0,0,1,1,1,0,1,0,1,0,1,1
2017-04-02,0,0,1,1,0,0,1,1,0,0,1,0,1,0,1,1
2017-04-03,0,0,1,1,0,0,1,1,1,0,1,0,1,0,1,1
2017-04-04,0,0,1,1,0,0,1,1,1,0,1,0,1,0,1,1
2017-04-05,0,0,1,1,0,0,1,1,0,0,1,0,1,0,1,1


In [43]:
df2 = df2.merge(each_state_count, how = 'left', on = ['application_date'])
df2 = df2.merge(each_state_nonzero, how = 'left', on = ['application_date'])
df2.head()

,application_date,case_count,segment,state_count,state_ASSAM_x,state_BIHAR_x,state_CHHATTISGARH_x,state_GUJARAT_x,state_HARYANA_x,state_JHARKHAND_x,state_KARNATAKA_x,state_KERALA_x,state_MADHYA PRADESH_x,state_MAHARASHTRA_x,state_ORISSA_x,state_PUNJAB_x,state_TAMIL NADU_x,state_TRIPURA_x,state_UTTAR PRADESH_x,state_WEST BENGAL_x,state_ASSAM_y,state_BIHAR_y,state_CHHATTISGARH_y,state_GUJARAT_y,state_HARYANA_y,state_JHARKHAND_y,state_KARNATAKA_y,state_KERALA_y,state_MADHYA PRADESH_y,state_MAHARASHTRA_y,state_ORISSA_y,state_PUNJAB_y,state_TAMIL NADU_y,state_TRIPURA_y,state_UTTAR PRADESH_y,state_WEST BENGAL_y
0,2017-04-01,897.0,2,16.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
1,2017-04-02,605.0,2,16.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
2,2017-04-03,2016.0,2,16.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
3,2017-04-04,2245.0,2,16.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0
4,2017-04-05,2415.0,2,16.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0


In [0]:
from fastai.tabular import add_datepart

add_datepart(df2, 'application_date', drop=False)

df2.drop('application_Elapsed', axis=1, inplace=True)

In [45]:
import holidays
ind_holidays = []

for date in holidays.India(years =[2017,2018,2019]).items():
  ind_holidays.append(str(date[0]))

print(ind_holidays, end=',')
#The below list does not make any sense! There is no festival holidays available!

['2017-01-14', '2017-01-26', '2017-08-15', '2017-10-02', '2017-05-01', '2017-12-25', '2018-01-14', '2018-01-26', '2018-08-15', '2018-10-02', '2018-05-01', '2018-12-25', '2019-01-14', '2019-01-26', '2019-08-15', '2019-10-02', '2019-05-01', '2019-12-25'],

In [46]:
# Refering calendarlabs.com
#['2017-01-01','2017-03-13','2017-03-28','2017-04-05','2017-04-14','2017-06-26','2017-08-07','2017-08-14','2017-08-25',
#'2017-09-04','2017-09-30','2017-10-19','2018-01-01','2018-03-18','2018-03-25','2018-03-30','2018-04-14','2018-06-16','2018-08-25',
#'2018-08-26','2018-10-19','2018-11-07','2019-01-01','2019-03-21','2019-04-06','2019-04-13','2019-04-19','2019-06-05','2019-08-24',
#'2019-09-02','2019-09-11','2019-10-08','2019-10-27'])
ind_holidays.append('2017-01-01')
ind_holidays.append('2017-03-13')
ind_holidays.append('2017-03-28')
ind_holidays.append('2017-04-05')
ind_holidays.append('2017-04-14')
ind_holidays.append('2017-06-26')
ind_holidays.append('2017-08-07')
ind_holidays.append('2017-08-14')
ind_holidays.append('2017-08-25')
ind_holidays.append('2017-09-04')
ind_holidays.append('2017-09-30')
ind_holidays.append('2017-10-19')
ind_holidays.append('2018-01-01')
ind_holidays.append('2018-03-18')
ind_holidays.append('2018-03-25')
ind_holidays.append('2018-03-30')
ind_holidays.append('2018-04-14')
ind_holidays.append('2018-06-16')
ind_holidays.append('2018-08-25')
ind_holidays.append('2018-08-26')
ind_holidays.append('2018-10-19')
ind_holidays.append('2018-11-07')
ind_holidays.append('2019-01-01')
ind_holidays.append('2019-03-21')
ind_holidays.append('2019-04-06')
ind_holidays.append('2019-04-13')
ind_holidays.append('2019-04-19')
ind_holidays.append('2019-06-05')
ind_holidays.append('2019-08-24')
ind_holidays.append('2019-09-02')
ind_holidays.append('2019-09-11')
ind_holidays.append('2019-10-08')
ind_holidays.append('2019-10-27')
ind_holidays

['2017-01-14',
 '2017-01-26',
 '2017-08-15',
 '2017-10-02',
 '2017-05-01',
 '2017-12-25',
 '2018-01-14',
 '2018-01-26',
 '2018-08-15',
 '2018-10-02',
 '2018-05-01',
 '2018-12-25',
 '2019-01-14',
 '2019-01-26',
 '2019-08-15',
 '2019-10-02',
 '2019-05-01',
 '2019-12-25',
 '2017-01-01',
 '2017-03-13',
 '2017-03-28',
 '2017-04-05',
 '2017-04-14',
 '2017-06-26',
 '2017-08-07',
 '2017-08-14',
 '2017-08-25',
 '2017-09-04',
 '2017-09-30',
 '2017-10-19',
 '2018-01-01',
 '2018-03-18',
 '2018-03-25',
 '2018-03-30',
 '2018-04-14',
 '2018-06-16',
 '2018-08-25',
 '2018-08-26',
 '2018-10-19',
 '2018-11-07',
 '2019-01-01',
 '2019-03-21',
 '2019-04-06',
 '2019-04-13',
 '2019-04-19',
 '2019-06-05',
 '2019-08-24',
 '2019-09-02',
 '2019-09-11',
 '2019-10-08',
 '2019-10-27']

In [47]:
start_date = datetime.datetime(year=2017, month=1, day=1)
end_date = datetime.datetime(year=2019, month=12, day=31)

pd.date_range(start_date, end_date)

DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08',
               '2017-01-09', '2017-01-10',
               ...
               '2019-12-22', '2019-12-23', '2019-12-24', '2019-12-25',
               '2019-12-26', '2019-12-27', '2019-12-28', '2019-12-29',
               '2019-12-30', '2019-12-31'],
              dtype='datetime64[ns]', length=1095, freq='D')

In [48]:
date_range = pd.DataFrame()
date_range['application_date'] = pd.date_range(start_date, end_date)
date_range.head() 

,application_date
0,2017-01-01
1,2017-01-02
2,2017-01-03
3,2017-01-04
4,2017-01-05


In [0]:
date_range['is_Holiday'] = [1 if str(val).split()[0] in ind_holidays else 0 for val in date_range['application_date']]

In [50]:
date_range.head(100)

,application_date,is_Holiday
0,2017-01-01,1
1,2017-01-02,0
2,2017-01-03,0
3,2017-01-04,0
4,2017-01-05,0
...,...,...
95,2017-04-06,0
96,2017-04-07,0
97,2017-04-08,0
98,2017-04-09,0


In [51]:
df1 = df1.merge(date_range, how = 'left', on = ['application_date'])
df2 = df2.merge(date_range, how = 'left', on = ['application_date'])
df1.head()

,application_date,case_count,segment,state_count,unique_state,unique_state_zone,state_ANDHRA PRADESH_x,state_ASSAM_x,state_BIHAR_x,state_CHHATTISGARH_x,state_DELHI_x,state_GUJARAT_x,state_HARYANA_x,state_JHARKHAND_x,state_KARNATAKA_x,state_KERALA_x,state_MADHYA PRADESH_x,state_MAHARASHTRA_x,state_ORISSA_x,state_PUNJAB_x,state_TAMIL NADU_x,state_TELANGANA_x,state_TRIPURA_x,state_UTTAR PRADESH_x,state_UTTARAKHAND_x,state_WEST BENGAL_x,state_zone_CENTRAL CHHATTISGARH_x,state_zone_CENTRAL MADHYA PRADESH_x,state_zone_EAST ASSAM_x,state_zone_EAST BIHAR_x,state_zone_EAST ORISSA_x,state_zone_EAST TRIPURA_x,state_zone_EAST UTTAR PRADESH_x,state_zone_EAST UTTARAKHAND_x,state_zone_EAST WEST BENGAL_x,state_zone_NORTH DELHI_x,state_zone_NORTH HARYANA_x,state_zone_NORTH PUNJAB_x,state_zone_SOUTH ANDHRA PRADESH_x,state_zone_SOUTH JHARKHAND_x,state_zone_SOUTH KARNATAKA_x,state_zone_SOUTH KERALA_x,state_zone_SOUTH ORISSA_x,state_zone_SOUTH TAMIL NADU_x,state_zone_SOUTH TELANGANA_x,state_zone_WEST GUJARAT_x,state_zone_WEST MAHARASHTRA_x,state_ANDHRA PRADESH_y,state_ASSAM_y,state_BIHAR_y,state_CHHATTISGARH_y,state_DELHI_y,state_GUJARAT_y,state_HARYANA_y,state_JHARKHAND_y,state_KARNATAKA_y,state_KERALA_y,state_MADHYA PRADESH_y,state_MAHARASHTRA_y,state_ORISSA_y,state_PUNJAB_y,state_TAMIL NADU_y,state_TELANGANA_y,state_TRIPURA_y,state_UTTAR PRADESH_y,state_UTTARAKHAND_y,state_WEST BENGAL_y,state_zone_CENTRAL CHHATTISGARH_y,state_zone_CENTRAL MADHYA PRADESH_y,state_zone_EAST ASSAM_y,state_zone_EAST BIHAR_y,state_zone_EAST ORISSA_y,state_zone_EAST TRIPURA_y,state_zone_EAST UTTAR PRADESH_y,state_zone_EAST UTTARAKHAND_y,state_zone_EAST WEST BENGAL_y,state_zone_NORTH DELHI_y,state_zone_NORTH HARYANA_y,state_zone_NORTH PUNJAB_y,state_zone_SOUTH ANDHRA PRADESH_y,state_zone_SOUTH JHARKHAND_y,state_zone_SOUTH KARNATAKA_y,state_zone_SOUTH KERALA_y,state_zone_SOUTH ORISSA_y,state_zone_SOUTH TAMIL NADU_y,state_zone_SOUTH TELANGANA_y,state_zone_WEST GUJARAT_y,state_zone_WEST MAHARASHTRA_y,application_Year,application_Month,application_Week,application_Day,application_Dayofweek,application_Dayofyear,application_Is_month_end,application_Is_month_start,application_Is_quarter_end,application_Is_quarter_start,application_Is_year_end,application_Is_year_start,is_Holiday
0,2017-04-01,299.0,1,83.0,20.0,21.0,6.0,4.0,3.0,1.0,1.0,8.0,4.0,2.0,3.0,1.0,1.0,13.0,3.0,5.0,7.0,2.0,2.0,7.0,2.0,8.0,1.0,1.0,4.0,3.0,1.0,2.0,7.0,2.0,8.0,1.0,4.0,5.0,6.0,2.0,3.0,1.0,2.0,7.0,2.0,8.0,13.0,0.0,2.0,1.0,0.0,1.0,4.0,0.0,2.0,1.0,0.0,1.0,4.0,3.0,4.0,2.0,1.0,1.0,0.0,1.0,6.0,0.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,6.0,1.0,0.0,4.0,0.0,2.0,1.0,0.0,2.0,2.0,1.0,4.0,4.0,2017,4,13,1,5,91,False,True,False,True,False,False,0
1,2017-04-03,42.0,1,83.0,20.0,21.0,6.0,4.0,3.0,1.0,1.0,8.0,4.0,2.0,3.0,1.0,1.0,13.0,3.0,5.0,7.0,2.0,2.0,7.0,2.0,8.0,1.0,1.0,4.0,3.0,1.0,2.0,7.0,2.0,8.0,1.0,4.0,5.0,6.0,2.0,3.0,1.0,2.0,7.0,2.0,8.0,13.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,2.0,1.0,3.0,0.0,0.0,0.0,0.0,1.0,4.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,4.0,1.0,0.0,3.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,2.0,2017,4,14,3,0,93,False,False,False,False,False,False,0
2,2017-04-04,23.0,1,83.0,20.0,21.0,6.0,4.0,3.0,1.0,1.0,8.0,4.0,2.0,3.0,1.0,1.0,13.0,3.0,5.0,7.0,2.0,2.0,7.0,2.0,8.0,1.0,1.0,4.0,3.0,1.0,2.0,7.0,2.0,8.0,1.0,4.0,5.0,6.0,2.0,3.0,1.0,2.0,7.0,2.0,8.0,13.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,3.0,2017,4,14,4,1,94,False,False,False,False,False,False,0
3,2017-04-05,1530.0,1,83.0,20.0,21.0,6.0,4.0,3.0,1.0,1.0,8.0,4.0,2.0,3.0,1.0,1.0,13.0,3.0,5.0,7.0,2.0,2.0,7.0,2.0,8.0,1.0,1.0,4.0,3.0,1.0,2.0,7.0,2.0,8.0,1.0,4.0,5.0,6.0,2.0,3.0,1.0,2.0,7.0,2.0,8.0,13.0,1.0,2.0,1.0,0.0,1.0,5.0,0.0,2.0,2.0,1.0,1.0,9.0,3.0,4.0,3.0,1.0,1.0,0.0,1.0,8.0,0.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,8.0,1.0,0.0,4.0,1.0,2.0,2.0,1.0,2.0,3.0,1.0,5.0,9.0,2017,4,14,5,2,95,False,False,False,False,False,False,1
4,2017-04-07,1341.0,1,83.0,20.0,21.0,6.0,4.0,3.0

In [52]:
df2.head()

,application_date,case_count,segment,state_count,state_ASSAM_x,state_BIHAR_x,state_CHHATTISGARH_x,state_GUJARAT_x,state_HARYANA_x,state_JHARKHAND_x,state_KARNATAKA_x,state_KERALA_x,state_MADHYA PRADESH_x,state_MAHARASHTRA_x,state_ORISSA_x,state_PUNJAB_x,state_TAMIL NADU_x,state_TRIPURA_x,state_UTTAR PRADESH_x,state_WEST BENGAL_x,state_ASSAM_y,state_BIHAR_y,state_CHHATTISGARH_y,state_GUJARAT_y,state_HARYANA_y,state_JHARKHAND_y,state_KARNATAKA_y,state_KERALA_y,state_MADHYA PRADESH_y,state_MAHARASHTRA_y,state_ORISSA_y,state_PUNJAB_y,state_TAMIL NADU_y,state_TRIPURA_y,state_UTTAR PRADESH_y,state_WEST BENGAL_y,application_Year,application_Month,application_Week,application_Day,application_Dayofweek,application_Dayofyear,application_Is_month_end,application_Is_month_start,application_Is_quarter_end,application_Is_quarter_start,application_Is_year_end,application_Is_year_start,is_Holiday
0,2017-04-01,897.0,2,16.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,2017,4,13,1,5,91,False,True,False,True,False,False,0
1,2017-04-02,605.0,2,16.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,2017,4,13,2,6,92,False,False,False,False,False,False,0
2,2017-04-03,2016.0,2,16.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,2017,4,14,3,0,93,False,False,False,False,False,False,0
3,2017-04-04,2245.0,2,16.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,2017,4,14,4,1,94,False,False,False,False,False,False,0
4,2017-04-05,2415.0,2,16.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,2017,4,14,5,2,95,False,False,False,False,False,False,1


In [0]:
#df1['segment'] = df1['segment'].astype('category')
#df2['segment'] = df2['segment'].astype('category')

In [54]:
df1_train=df1[df1['case_count'].isnull()==False].copy()
df1_test=df1[df1['case_count'].isnull()==True].copy()
df2_train=df2[df2['case_count'].isnull()==False].copy()
df2_test=df2[df2['case_count'].isnull()==True].copy()

print(df1_train.shape,df1_test.shape,df2_train.shape,df2_test.shape)


(806, 101) (87, 101) (844, 49) (93, 49)


In [0]:
from xgboost import XGBRFRegressor
from sklearn.model_selection import StratifiedKFold,train_test_split
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

In [56]:
X1,Y1=df1_train.drop(['application_date','case_count'],axis=1),df1_train['case_count']
X1test=df1_test.drop(['application_date','case_count'],axis=1)

X2,Y2=df2_train.drop(['application_date','case_count'],axis=1),df2_train['case_count']
X2test=df2_test.drop(['application_date','case_count'],axis=1)
print(X1.shape,X1test.shape,X2.shape,X2test.shape)
X1_train,X1_val,Y1_train,Y1_val = train_test_split(X1,Y1,test_size=0.25,random_state = 2020)
X2_train,X2_val,Y2_train,Y2_val = train_test_split(X2,Y2,test_size=0.25,random_state = 2020)

(806, 99) (87, 99) (844, 47) (93, 47)


In [57]:
LGBMerror1=[]
y_pred_LGBM1=[]
fold=StratifiedKFold(n_splits=3,shuffle=True,random_state=80402)
i=1
for train_index, test_index in fold.split(X1,Y1):
    X1_train, X1_test = X1.iloc[train_index], X1.iloc[test_index]
    Y1_train, Y1_test = Y1[train_index], Y1[test_index]
    model=LGBMRegressor(boosting_type='gbdt',n_estimators=3000,learning_rate = 0.01, objective='regression', colsample_bytree=0.3)
    model.fit(X1_train,Y1_train,eval_set=[(X1_test, Y1_test)],eval_metric='mae', early_stopping_rounds=5000, verbose=2000)
    preds=model.predict(X1_test)[:]
    print("LGBMerror: ",mean_absolute_error(Y1_test,preds))
    LGBMerror1.append(mean_absolute_error(Y1_test,preds))
    p = model.predict(X1test)[:]
    i=i+1
    y_pred_LGBM1.append(p)

Training until validation scores don't improve for 5000 rounds.
[2000]	valid_0's l2: 2.24886e+06	valid_0's l1: 657.116
Did not meet early stopping. Best iteration is:
[1146]	valid_0's l2: 2.19063e+06	valid_0's l1: 636.984
LGBMerror:  636.9841130845737
Training until validation scores don't improve for 5000 rounds.
[2000]	valid_0's l2: 1.6094e+06	valid_0's l1: 697.403
Did not meet early stopping. Best iteration is:
[454]	valid_0's l2: 1.28715e+06	valid_0's l1: 570.137
LGBMerror:  570.1370911446868
Training until validation scores don't improve for 5000 rounds.
[2000]	valid_0's l2: 2.27249e+06	valid_0's l1: 598.898
Did not meet early stopping. Best iteration is:
[931]	valid_0's l2: 2.21789e+06	valid_0's l1: 577.055
LGBMerror:  577.0546705674464


In [58]:
LGBMerror2=[]
y_pred_LGBM2=[]
fold=StratifiedKFold(n_splits=2,shuffle=True,random_state=80402)
i=1
for train_index, test_index in fold.split(X2,Y2):
    X2_train, X2_test = X2.iloc[train_index], X2.iloc[test_index]
    Y2_train, Y2_test = Y2[train_index], Y2[test_index]
    model=LGBMRegressor(boosting_type='gbdt',n_estimators=3000,learning_rate = 0.01, objective='regression', colsample_bytree=0.3)
    model.fit(X2_train,Y2_train,eval_set=[(X2_test, Y2_test)],eval_metric='mae', early_stopping_rounds=5000, verbose=2000)
    preds=model.predict(X2_test)[:]
    print("LGBMerror: ",mean_absolute_error(Y2_test,preds))
    LGBMerror2.append(mean_absolute_error(Y2_test,preds))
    p = model.predict(X2test)[:]
    i=i+1
    y_pred_LGBM2.append(p)

Training until validation scores don't improve for 5000 rounds.
[2000]	valid_0's l2: 8.86547e+06	valid_0's l1: 2248.7
Did not meet early stopping. Best iteration is:
[2998]	valid_0's l2: 8.39768e+06	valid_0's l1: 2175.57
LGBMerror:  2175.572018935278
Training until validation scores don't improve for 5000 rounds.
[2000]	valid_0's l2: 9.48423e+06	valid_0's l1: 2200.93
Did not meet early stopping. Best iteration is:
[2999]	valid_0's l2: 9.09579e+06	valid_0's l1: 2133.02
LGBMerror:  2133.0213068729113


In [59]:
np.mean(LGBMerror1,0)

594.7252915989023

In [0]:
submission1 = submission[submission['segment'] == 1]
submission2 = submission[submission['segment'] == 2]

In [0]:
submission1['case_count'] = np.abs(np.mean(y_pred_LGBM1,0))
submission2['case_count'] = np.abs(np.mean(y_pred_LGBM2,0))

In [62]:
final_submission = pd.concat([submission1,submission2])
final_submission

,id,application_date,segment,case_count
0,1,2019-07-06,1,166.665434
1,2,2019-07-07,1,119.808557
2,3,2019-07-08,1,45.939295
3,4,2019-07-09,1,119.699016
4,5,2019-07-10,1,62.902789
...,...,...,...,...
175,176,2019-10-20,2,8446.214970
176,177,2019-10-21,2,15013.837328
177,178,2019-10-22,2,16228.443993
178,179,2019-10-23,2,16739.641541


In [63]:
final_submission.to_csv("/content/drive/My Drive/LTFS/final_submission_LGBM.csv", index= False)
final_submission.shape

(180, 4)

[0]	validation_0-mae:595.079
Will train until validation_0-mae hasn't improved in 5000 rounds.
XGError:  595.0788595242128
[0]	validation_0-mae:531.455
Will train until validation_0-mae hasn't improved in 5000 rounds.
XGError:  531.4552548809122
[0]	validation_0-mae:568.949
Will train until validation_0-mae hasn't improved in 5000 rounds.
XGError:  568.94928798035


ValueError: ignored